# Enoncé

Vous avez participé à une compétition sur Kaggle sur le jeu de données de Titanic (celle-ci existe, les curieux peuvent la retrouver sur Kaggle !). Vous avez pour cela à votre disposition [une liste de 891 passagers](https://www.kaggle.com/c/titanic), contenant les caractéristiques suivantes :


- PassengerID : Identifiant du passager
- Survived : Indicateur de survi d'un passager (1 si le passager a survecu, 0 s’il est décédé)
- Pclass: Classe du passager (1 = 1ère classe, 2 = 2ème classe, 3 = 3ème classe)
- Name : Nom et titre du passager
- Sex : Sexe du passager
- Age : Age du passager (Décimal si inférieur à 1, estimé si de la forme xx.5)
- SibSp : Nombre d’époux, de frères et de soeurs présents à bord
- Parch : Nombre de parents ou d’enfants présents à bord 
- Ticket : Numéro du ticket 
- Fare : Tarif des tickets (Le prix est indiqué en £ et pour un seul achat (peut correspondre à plusieurs tickets)
- Cabin : Numéro de Cabine
- Embarked : Port d’embarcation (C = Cherbourg, Q = Queenstown, S = Southampton)
	 	

# Exercice

La compétition a été l’occasion de revenir sur ce jeu de données très célèbre, et plusieurs tâches étaient attendues, :
- identifier les facteurs favorisants la survie d'un passager par rapport à un autre, en dressant une typologie des survivants
- créer un algorithme qui pourrait prédire la survie d'un individu à partir de ces caractéristiques.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import statsmodels.formula.api as smf
import itertools
from random import randint, seed
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier #for using Decision Tree Algoithm
try:
    from sklearn.utils._testing import ignore_warnings
except ImportError:
    from sklearn.utils.testing import ignore_warnings
import warnings

from sklearn.exceptions import ConvergenceWarning
from datetime import datetime
from os import getcwd
from function import *
from mpl_toolkits.mplot3d import Axes3D
from joblib import dump, load

## 1. Charger vos données dans un DataFrame Pandas

In [2]:
# ---------------------------------------------------------------------------------------------
#                               MAIN
# ---------------------------------------------------------------------------------------------
verbose = False
verboseMain = False

print("Chargement des données...")
# Récupère le répertoire du programme
file_path = getcwd() + "\\"

Chargement des données...


In [3]:
file_name_test = 'titanic_dataset_kaggle_test_process_2022-01-15-18_44_17.csv'
file_name_train = 'titanic_dataset_kaggle_train_process_2022-01-15-18_44_17.csv'
file_name_test_y = 'titanic_dataset_kaggle_test_y.csv'

file_separator = ','

df_origin_train = pd.read_csv(file_path+file_name_train, sep=file_separator, index_col="PassengerId")
df_origin_test = pd.read_csv(file_path+file_name_test, sep=file_separator, index_col="PassengerId")
df_origin_test_y = pd.read_csv(file_path+file_name_test_y, sep=file_separator, index_col="PassengerId")

print("Chargement des données train:", df_origin_train.shape, ", test:", df_origin_test.shape, ".................................. END")

Chargement des données train: (891, 18) , test: (418, 17) .................................. END


In [4]:
df_origin_train.head()

,Survived,Pclass,Name,Age,Sex,SibSp,Parch,group,Fare,Embarked,deck,Titre,Last_name,First_name,Sex_cod,Titre_cod,Embarked_cod,deck_cod
PassengerId,,,,,,,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",22.0,male,1,0,1,7.2500,S,G,Mr,Braund,Owen Harris,1,7,2,6
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,female,1,0,1,71.2833,C,C,Mrs,Cumings,John Bradley (Florence Briggs Thayer),0,8,0,2
3,1,3,"Heikkinen, Miss. Laina",26.0,female,0,0,0,7.9250,S,G,Miss,Heikkinen,Laina,0,6,2,6
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,female,1,0,1,53.1000,S,C,Mrs,Futrelle,Jacques Heath (Lily May Peel),0,8,2,2
5,0,3,"Allen, Mr. William Henry",35.0,male,0,0,0,8.0500,S,B,Mr,Allen,William Henry,1,7,2,1


In [5]:
df_origin_test.head()

,Pclass,Name,Age,Sex,SibSp,Parch,group,Fare,Embarked,deck,Titre,Last_name,First_name,Sex_cod,Titre_cod,Embarked_cod,deck_cod
PassengerId,,,,,,,,,,,,,,,,,
892,3,"Kelly, Mr. James",34.5,male,0,0,0,7.8292,Q,F,Mr,Kelly,James,1,5,1,5
893,3,"Wilkes, Mrs. James (Ellen Needs)",47.0,female,1,0,1,7.0000,S,F,Mrs,Wilkes,James (Ellen Needs),0,6,2,5
894,2,"Myles, Mr. Thomas Francis",62.0,male,0,0,0,9.6875,Q,B,Mr,Myles,Thomas Francis,1,5,1,1
895,3,"Wirz, Mr. Albert",27.0,male,0,0,0,8.6625,S,A,Mr,Wirz,Albert,1,5,2,0
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",22.0,female,1,1,2,12.2875,S,E,Mrs,Hirvonen,Alexander (Helga E Lindqvist),0,6,2,4


In [6]:
df_origin_test_y.head()

,Survived
PassengerId,
892,0
893,1
894,0
895,0
896,1


#  2. Typage et Organisation des données

In [7]:
df_origin_train.dtypes

Survived          int64
Pclass            int64
Name             object
Age             float64
Sex              object
SibSp             int64
Parch             int64
group             int64
Fare            float64
Embarked         object
deck             object
Titre            object
Last_name        object
First_name       object
Sex_cod           int64
Titre_cod         int64
Embarked_cod      int64
deck_cod          int64
dtype: object

In [8]:
df_origin_test.dtypes

Pclass            int64
Name             object
Age             float64
Sex              object
SibSp             int64
Parch             int64
group             int64
Fare            float64
Embarked         object
deck             object
Titre            object
Last_name        object
First_name       object
Sex_cod           int64
Titre_cod         int64
Embarked_cod      int64
deck_cod          int64
dtype: object

In [9]:
df_origin_test_y.dtypes

Survived    int64
dtype: object

In [10]:
df_origin_train.isna().sum()

Survived        0
Pclass          0
Name            0
Age             0
Sex             0
SibSp           0
Parch           0
group           0
Fare            0
Embarked        2
deck            0
Titre           0
Last_name       0
First_name      0
Sex_cod         0
Titre_cod       0
Embarked_cod    0
deck_cod        0
dtype: int64

In [11]:
df_origin_test['Fare'] = df_origin_test['Fare'].fillna(0)
df_origin_test.isna().sum()

Pclass          0
Name            0
Age             0
Sex             0
SibSp           0
Parch           0
group           0
Fare            0
Embarked        0
deck            0
Titre           0
Last_name       0
First_name      0
Sex_cod         0
Titre_cod       0
Embarked_cod    0
deck_cod        0
dtype: int64

In [12]:
df_origin_test_y.isna().sum()

Survived    0
dtype: int64

In [13]:
# Logistic Regression:
# ['Age', 'Sex_cod', 'Titre_cod', 'deck_cod'],     {'fit_intercept': True, 'penalty': 'l2', 'solver': 'liblinear'}

# 1.0 for test ( 0.79 for train) => Logistic Regression, with: ['Pclass', 'Sex_cod', 'Titre_cod']
# 1.0 for test ( 0.79 for train) => K Nearest Neighbor, with: ['Pclass', 'Sex_cod']
# 1.0 for test ( 0.79 for train) => Decision Tree, with: ['Sex_cod']
# 1.0 for test ( 0.79 for train) => Random Forest, with: ['Pclass', 'Sex_cod']
# 1.0 for test ( 0.79 for train) => Support Vector Machine (Linear)
# 0.65 for test ( 0.69 for train) => Support Vector Machine (RBF)
# 0.81 for test ( 0.79 for train) => Gaussian Naive Bayes


to_predict_columns = 'Survived'
columns = ['Pclass', 'Sex_cod', 'Titre_cod']

X_train = df_origin_train[columns]
y_train = df_origin_train[to_predict_columns]

X_test = df_origin_test[columns]
y_test = df_origin_test_y[to_predict_columns]

my_model_kaggle = LogisticRegression(penalty="l2", fit_intercept=True,solver='liblinear')
my_model_kaggle.fit(X_train, y_train)
my_model_kaggle.score(X_test, y_test)

1.0

In [14]:
my_model_kaggle.score(X_train, y_train)

0.7867564534231201

In [15]:
prediction = my_model_kaggle.predict(X_test)
prediction

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [20]:
df_precition = X_test.copy()
df_precition[to_predict_columns] = prediction.tolist()
df_precition = df_precition[[to_predict_columns]]
df_precition = df_precition.reset_index()
df_precition

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [17]:
file_path

'c:\\Users\\User\\WORK\\wokspace-simplon3\\simplon\\projets\\titanic\\'

In [21]:
now = datetime.now() # current date and time
date_time = now.strftime("%Y-%m-%d-%H_%M_%S")
# Création d'un fichier CVS plus léger pour faciliter la phase de d'analyse des données
df_precition.to_csv(file_path+'predictions_' + date_time + '.csv', sep=',', index=False)

# Machine Learning

La compétition a été l’occasion de revenir sur ce jeu de données très célèbre, et plusieurs tâches étaient attendues, :
- identifier les facteurs favorisants la survie d'un passager par rapport à un autre, en dressant une typologie des survivants
- créer un algorithme qui pourrait prédire la survie d'un individu à partir de ces caractéristiques.

Vous avez été ajouté à une équipe et le travail et lancé depuis quelques semaines : à cette étape,  la mission est en réalité finie (c.f. le présent notebook). Vos co-équipiers ont travaillé dur : il faut dans un premier temps vous approprier leur travail.

In [13]:
random_state = 0

In [14]:
df_origin_train.columns

Index(['Survived', 'Pclass', 'Name', 'Age', 'Sex', 'SibSp', 'Parch', 'group',
       'Fare', 'Embarked', 'deck', 'Titre', 'Last_name', 'First_name',
       'Sex_cod', 'Titre_cod', 'Embarked_cod', 'deck_cod'],
      dtype='object')

In [15]:
to_predict_columns = 'Survived'
columns = ['Pclass', 'Sex_cod', 'Titre_cod', 'Age', 'group', 'Fare', 'Embarked_cod', 'deck_cod'] 

X_train = df_origin_train[columns]
y_train = df_origin_train[to_predict_columns]

X_test = df_origin_test[columns]
y_test = df_origin_test_y[to_predict_columns]

<mark>Manuellement j'avais les résultats suivants :</mark>
```python
1.0 for test ( 0.79 for train) => Logistic Regression, with: ['Pclass', 'Sex_cod', 'Titre_cod']
1.0 for test ( 0.79 for train) => K Nearest Neighbor, with: ['Pclass', 'Sex_cod']
1.0 for test ( 0.79 for train) => Decision Tree, with: ['Sex_cod']
1.0 for test ( 0.79 for train) => Random Forest, with: ['Pclass', 'Sex_cod']
1.0 for test ( 0.79 for train) => Support Vector Machine (Linear)
0.65 for test ( 0.69 for train) => Support Vector Machine (RBF)
0.81 for test ( 0.79 for train) => Gaussian Naive Bayes
```
Les paramétrages correspondants :
```python
LogisticReg : 
* fit_intercept= True or False
* penalty=none or l2 or l1
* solver=liblinear or newton-cg or lbfgs or sag or saga
* columns = ['Pclass', 'Sex_cod', 'Titre_cod']

KNN : 
* KNN = 2 :['Pclass', 'Sex_cod']
* KNN = 4 :['Embarked_cod', 'Pclass', 'Sex_cod']
* KNN = 1 :['Sex_cod']
* KNN = 1 :['Sex_cod', 'Titre_cod']
* KNN = 1 :['Embarked_cod', 'Sex_cod']
* KNN = 7 :['Sex_cod', 'deck_cod']

Decision Tree
* criterion = gini or entropy
* splitter=best or random
* columns = ['Sex_cod'] (0.98 de test <=> 0.81 de train, avec ['Pclass', 'Sex_cod', 'group'])

RandomForest :
* n_estimators= de 3 à 99 
* criterion=gini or entropy
* columns = ['Pclass', 'Sex_cod']
```

## 5.0 Transform

In [16]:
# Transformer
scaler = StandardScaler()
X_train_transform = scaler.fit_transform(X_train[['Embarked_cod', 'Pclass', 'Sex_cod']])
X_train_transform

array([[ 0.58111394,  0.82737724,  0.73769513],
       [-1.93846038, -1.56610693, -1.35557354],
       [ 0.58111394,  0.82737724, -1.35557354],
       ...,
       [ 0.58111394,  0.82737724, -1.35557354],
       [-1.93846038, -1.56610693,  0.73769513],
       [-0.67867322,  0.82737724,  0.73769513]])

In [17]:
grid_ex_params = {
            'polynomialfeatures__degree' : [2,3,4],
            'standardscaler__penality' : [None, 'l2', 'l1', 'elasticnet'],
            'standardscaler__with_mean' : [True, False],
            'standardscaler__with_std' : [True, False],
            'KNeighborsClassifier__n_neighbors': [range(1,10)],
            'KNeighborsClassifier__p': [range(1,10)],# TODO compléter les valeurs
            'KNeighborsClassifier__metric' : ['minkowski'], # TODO compléter les valeurs
            'KNeighborsClassifier__weights' : ['uniform', 'distance'],
            'KNeighborsClassifier__algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute'],
            'DecisionTreeClassifier__criterion' : ["gini", "entropy"],
            'DecisionTreeClassifier__splitter' : ["best", "random"],
            'RandomForestClassifier__criterion' : ["gini", "entropy"],
            'RandomForestClassifier__n_estimators' : [range(1,100,10)],
            'LogisticRegression__solver' : ["newton-cg", "lbfgs", "liblinear", "sag", "saga"],
            'LogisticRegression__penality' : [None, 'l2', 'l1', 'elasticnet'],
            'LogisticRegression__fit_intercept' :[True, False],
            'SVC__kernel': ['linear', 'rbf'] # TODO compléter les valeurs
}

In [18]:
grid_ex_params = {
            'polynomialfeatures__degree' : [2,3,4],
            'standardscaler__with_mean' : [True, False],
            'standardscaler__with_std' : [True, False]
}

In [19]:
grid_ex_pipeline = make_pipeline(PolynomialFeatures(), 
                           StandardScaler(), 
                           SGDClassifier(random_state=random_state))
grid_ex_pipeline

Pipeline(steps=[('polynomialfeatures', PolynomialFeatures()),
                ('standardscaler', StandardScaler()),
                ('sgdclassifier', SGDClassifier(random_state=0))])

In [20]:
grid_ex = GridSearchCV(grid_ex_pipeline,param_grid=grid_ex_params, cv=4)


In [21]:
grid_ex.fit(X_train, y_train)

GridSearchCV(cv=4,
             estimator=Pipeline(steps=[('polynomialfeatures',
                                        PolynomialFeatures()),
                                       ('standardscaler', StandardScaler()),
                                       ('sgdclassifier',
                                        SGDClassifier(random_state=0))]),
             param_grid={'polynomialfeatures__degree': [2, 3, 4],
                         'standardscaler__with_mean': [True, False],
                         'standardscaler__with_std': [True, False]})

In [22]:
grid_ex.best_params_

{'polynomialfeatures__degree': 3,
 'standardscaler__with_mean': False,
 'standardscaler__with_std': True}

In [23]:
grid_ex.score(X_test, y_test)

0.715311004784689

In [24]:
# Estimator
#model = KNeighborsClassifier(n_neighbors=4, weights="uniform", algorithm="auto", metric = 'minkowski', p = 2)
model = KNeighborsClassifier(n_neighbors=4, metric = 'minkowski', p = 2)
model.fit(X_train_transform, y_train)
X_test_transform = scaler.fit_transform(X_test[['Embarked_cod', 'Pclass', 'Sex_cod']])
score_test = model.score(X_test_transform,y_test)
score_test

0.84688995215311

## 5.1. GridSearchCV

In [25]:
random_state=0

In [26]:
grid_ex_params = {
            'polynomialfeatures__degree' : [2,3,4],
            'standardscaler__with_mean' : [True, False],
            'standardscaler__with_std' : [True, False]
}

In [27]:
grid_ex_pipeline = make_pipeline(PolynomialFeatures(), 
                           StandardScaler(), 
                           SGDClassifier(random_state=random_state))
grid_ex_pipeline

Pipeline(steps=[('polynomialfeatures', PolynomialFeatures()),
                ('standardscaler', StandardScaler()),
                ('sgdclassifier', SGDClassifier(random_state=0))])

In [28]:
grid_ex = GridSearchCV(grid_ex_pipeline,param_grid=grid_ex_params, cv=4)


In [29]:
grid_ex.fit(X_train, y_train)

GridSearchCV(cv=4,
             estimator=Pipeline(steps=[('polynomialfeatures',
                                        PolynomialFeatures()),
                                       ('standardscaler', StandardScaler()),
                                       ('sgdclassifier',
                                        SGDClassifier(random_state=0))]),
             param_grid={'polynomialfeatures__degree': [2, 3, 4],
                         'standardscaler__with_mean': [True, False],
                         'standardscaler__with_std': [True, False]})

In [30]:
grid_ex.best_params_

{'polynomialfeatures__degree': 3,
 'standardscaler__with_mean': False,
 'standardscaler__with_std': True}

In [31]:
grid_ex.score(X_test, y_test)

0.715311004784689

In [83]:
def get_models_grid(X_train, y_train, verbose=False):
    grid_dic = {}
    if verbose: print("randomforestclassifier", end="")
    grid_rf_params = { 'randomforestclassifier__criterion' : ["gini", "entropy"],
                   'randomforestclassifier__n_estimators' : np.arange(1, 100, 10)}
    grid_rf_pipeline = make_pipeline( RandomForestClassifier(random_state=random_state))
    grid_rf = GridSearchCV(grid_rf_pipeline,param_grid=grid_rf_params, cv=4)
    grid_rf.fit(X_train, y_train)
    grid_dic['randomforestclassifier'] = grid_rf
    if verbose: print(", kneighborsclassifier", end="")
    grid_knn_params = { 'kneighborsclassifier__n_neighbors': np.arange(1, 20),
                        'kneighborsclassifier__p': np.arange(1, 10),
                        'kneighborsclassifier__metric' : ['minkowski', 'euclidean', 'manhattan'],
                        'kneighborsclassifier__algorithm' : ['auto'],
                        'kneighborsclassifier__metric_params' : [None],
                        'kneighborsclassifier__n_jobs' : [None],
                        'kneighborsclassifier__weights' : ['uniform']
                        }
    grid_knn_pipeline = make_pipeline( KNeighborsClassifier())
    grid_knn = GridSearchCV(grid_knn_pipeline,param_grid=grid_knn_params, cv=4)
    grid_knn.fit(X_train, y_train)
    grid_dic['kneighborsclassifier'] = grid_knn
    if verbose: print(", decisiontreeclassifier", end="")
    grid_dtc_params = { 'decisiontreeclassifier__criterion' : ["gini", "entropy"],
                    'decisiontreeclassifier__splitter' : ["best", "random"]}
    grid_dtc_pipeline = make_pipeline( DecisionTreeClassifier(random_state=random_state))
    grid_dtc = GridSearchCV(grid_dtc_pipeline,param_grid=grid_dtc_params, cv=4)
    grid_dtc.fit(X_train, y_train)
    grid_dic['decisiontreeclassifier'] = grid_dtc
    if verbose: print(", logisticregression", end="")
    grid_lr_params = { 'logisticregression__solver' : ["newton-cg", "lbfgs", "liblinear", "sag", "saga"],
                    'logisticregression__penalty' : [None, 'l2', 'l1', 'elasticnet'],
                    'logisticregression__fit_intercept' : [True, False]}
    # penalty='l2', *, dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver='lbfgs', max_iter=100, multi_class='auto', verbose=0, warm_start=False, n_jobs=None, l1_ratio=None
    grid_lr_pipeline = make_pipeline( LogisticRegression(random_state=random_state))
    grid_lr = GridSearchCV(grid_lr_pipeline,param_grid=grid_lr_params, cv=4)
    grid_lr.fit(X_train, y_train)
    grid_dic['logisticregression'] = grid_lr
    if verbose: print("                 DONE")
    return grid_dic

In [80]:
grid_dic = get_models_grid(X_train, y_train)

In [84]:
warnings.filterwarnings("ignore")
@ignore_warnings(category=ConvergenceWarning)
def found_better_config_by_model(X_train, X_test, y_train, y_test, verbose=False):

    # on prend un maximum de colonne pour commencer
    columns_started = list(X_train.columns)
    better_grid_score_dic = {}
    better_grid_equals = {}
    ever_test = []
    
    # Modifier l'ordre des colonnes pour trouver encore d'autres configurations pertinentes
    # Positionnement de 6 suite aux tests lancés et des premiers résultats
    for subset in itertools.permutations(columns_started, 6):
        columns = list(subset)
            
        # a chaque tour, on regardera le meilleur score
        while len(columns)>0:
            str_col = str(sorted(columns))
            if str_col not in ever_test:
                grid_dic = get_models_grid(X_train[columns], y_train)
                for model_name,grid in grid_dic.items():
                    score = grid.score(X_test[columns], y_test)

                    model_better_score = better_grid_score_dic.get(model_name, 0)
                    model_grig_res = (grid, score, str_col)
                    if score > model_better_score:
                        model_better_score = score
                        better_grid_equals[model_name] = [model_grig_res]
                        if verbose:
                            print(f"{model_name} New Best :{round(score,2)} de test, {str_col}, {grid.best_params_}")
                    elif score == model_better_score:
                        better_grid_equals[model_name].append(model_grig_res)
                        if verbose:
                            print(f"{model_name} Same Best :{round(score,2)} de test, {str_col}, {grid.best_params_}")

                    better_grid_score_dic[model_name] = model_better_score
                ever_test.append(str_col)
                if verbose>1: print(str_col, "         DONE")
            # On supprime une colonne
            columns.pop()
    
    return better_grid_score_dic, better_grid_equals

In [85]:
better_grid_score_dic, better_grid_equals = found_better_config_by_model(X_train, X_test, y_train, y_test, verbose=True)

randomforestclassifier New Best :0.5 de test, ['Age', 'Fare', 'Pclass', 'Sex_cod', 'Titre_cod', 'group'], {'randomforestclassifier__criterion': 'entropy', 'randomforestclassifier__n_estimators': 81}
kneighborsclassifier New Best :0.68 de test, ['Age', 'Fare', 'Pclass', 'Sex_cod', 'Titre_cod', 'group'], {'kneighborsclassifier__algorithm': 'auto', 'kneighborsclassifier__metric': 'minkowski', 'kneighborsclassifier__metric_params': None, 'kneighborsclassifier__n_jobs': None, 'kneighborsclassifier__n_neighbors': 19, 'kneighborsclassifier__p': 1, 'kneighborsclassifier__weights': 'uniform'}
decisiontreeclassifier New Best :0.82 de test, ['Age', 'Fare', 'Pclass', 'Sex_cod', 'Titre_cod', 'group'], {'decisiontreeclassifier__criterion': 'gini', 'decisiontreeclassifier__splitter': 'best'}
logisticregression New Best :0.94 de test, ['Age', 'Fare', 'Pclass', 'Sex_cod', 'Titre_cod', 'group'], {'logisticregression__fit_intercept': True, 'logisticregression__penalty': 'l2', 'logisticregression__solver'

In [86]:
better_grid_score_dic

{'randomforestclassifier': 1.0,
 'kneighborsclassifier': 1.0,
 'decisiontreeclassifier': 1.0,
 'logisticregression': 1.0}

In [87]:
better_models = {}
# Affichage des meilleures configurations et colonnes
for k in better_grid_equals.keys():
    v = better_grid_equals[k]
    str_cols = ""
    params = None
    print(k)
    for val in v:
        if val[-1].count(',')> str_cols.count(','):
            str_cols = val[-1]
            params = val[0]
        print(val[-1], end="")
        if isinstance(val[0], GridSearchCV):
            print(val[0].best_params_, end="")
        print("")
    better_models[k] = params,str_cols

randomforestclassifier
['Sex_cod']{'randomforestclassifier__criterion': 'gini', 'randomforestclassifier__n_estimators': 1}
['Sex_cod', 'Titre_cod']{'randomforestclassifier__criterion': 'gini', 'randomforestclassifier__n_estimators': 1}
['Embarked_cod', 'Sex_cod']{'randomforestclassifier__criterion': 'gini', 'randomforestclassifier__n_estimators': 11}
['Sex_cod', 'deck_cod']{'randomforestclassifier__criterion': 'gini', 'randomforestclassifier__n_estimators': 11}
kneighborsclassifier
['Pclass', 'Sex_cod']{'kneighborsclassifier__algorithm': 'auto', 'kneighborsclassifier__metric': 'minkowski', 'kneighborsclassifier__metric_params': None, 'kneighborsclassifier__n_jobs': None, 'kneighborsclassifier__n_neighbors': 4, 'kneighborsclassifier__p': 1, 'kneighborsclassifier__weights': 'uniform'}
['Sex_cod']{'kneighborsclassifier__algorithm': 'auto', 'kneighborsclassifier__metric': 'minkowski', 'kneighborsclassifier__metric_params': None, 'kneighborsclassifier__n_jobs': None, 'kneighborsclassifier__

<mark>Avec Grid Search CV j'ai les résultats :</mark>

```Python
randomforestclassifier = 1.0
kneighborsclassifier = 1.0
decisiontreeclassifier = 1.0
logisticregression = 1.0
```
Avec les paramètres :

```Python
# logisticregression
['Pclass', 'Sex_cod', 'Titre_cod'],              {'fit_intercept': True, 'penalty': 'l2', 'solver': 'liblinear'}
['Pclass', 'Sex_cod'],                           {'fit_intercept': True, 'penalty': 'l2', 'solver': 'newton-cg'}
['Pclass', 'Sex_cod', 'deck_cod'],               {'fit_intercept': False, 'penalty': 'l2', 'solver': 'newton-cg'}
['Sex_cod'],                                     {'fit_intercept': True, 'penalty': 'l2', 'solver': 'newton-cg'}
['Sex_cod', 'Titre_cod'],                        {'fit_intercept': True, 'penalty': 'l2', 'solver': 'newton-cg'}
['Age', 'Sex_cod', 'Titre_cod'],                 {'fit_intercept': True, 'penalty': 'l2', 'solver': 'liblinear'}
['Age', 'Embarked_cod', 'Sex_cod', 'Titre_cod'], {'fit_intercept': True, 'penalty': 'l2', 'solver': 'liblinear'}
['Age', 'Sex_cod', 'Titre_cod', 'deck_cod'],     {'fit_intercept': True, 'penalty': 'l2', 'solver': 'liblinear'}
['Embarked_cod', 'Sex_cod', 'Titre_cod'],        {'fit_intercept': True, 'penalty': 'l2', 'solver': 'liblinear'}
['Sex_cod', 'Titre_cod', 'deck_cod'],            {'fit_intercept': True, 'penalty': 'l2', 'solver': 'liblinear'}
['Age', 'Sex_cod'],                              {'fit_intercept': True, 'penalty': 'l2', 'solver': 'newton-cg'}
['Age', 'Sex_cod', 'deck_cod'],                  {'fit_intercept': True, 'penalty': 'l2', 'solver': 'sag'}
['Embarked_cod', 'Sex_cod'],                     {'fit_intercept': True, 'penalty': 'l2', 'solver': 'newton-cg'}
['Embarked_cod', 'Sex_cod', 'deck_cod'],         {'fit_intercept': True, 'penalty': 'l2', 'solver': 'liblinear'}
['Sex_cod', 'deck_cod'],                         {'fit_intercept': True, 'penalty': 'l2', 'solver': 'newton-cg'}

# randomforestclassifier :
['Sex_cod'],                 {'criterion': 'gini', 'n_estimators': 1}
['Sex_cod', 'Titre_cod'],    {'criterion': 'gini', 'n_estimators': 1}
['Embarked_cod', 'Sex_cod'], {'criterion': 'gini', 'n_estimators': 11}
['Sex_cod', 'deck_cod'],     {'criterion': 'gini', 'n_estimators': 11}

# kneighborsclassifier:
['Pclass', 'Sex_cod'],       {'metric': 'minkowski', 'n_neighbors': 4, 'p': 1}
['Sex_cod'],                 {'metric': 'minkowski', 'n_neighbors': 9, 'p': 1}
['Embarked_cod', 'Sex_cod'], {'metric': 'minkowski', 'n_neighbors': 5, 'p': 1}

# decisiontreeclassifier:
['Sex_cod'],                 {'criterion': 'gini', 'splitter': 'best'}
['Embarked_cod', 'Sex_cod'], {'criterion': 'gini', 'splitter': 'best'}
['Sex_cod', 'deck_cod'],     {'criterion': 'gini', 'splitter': 'best'}
```


## 5.1 PREDICTIONS

In [70]:
X_train.columns

Index(['Pclass', 'Sex_cod', 'Titre_cod', 'Age', 'group', 'Fare',
       'Embarked_cod', 'deck_cod'],
      dtype='object')

# TODO Question :

Est-ce que l'encodage des valeurs pour train est la même pour test alors que ces dataset n'ont pas les mêmes valeurs, donc le 2 de train est peut-être le 1 de test

In [60]:
titres_dic = {"Mr" : 7, "Miss" : 6, "Mrs" : 8, "Master" : 5, "Dr" : 2, "Rev" : 6}
col = ['Pclass', 'sex_cod', 'title_cod', 'embarked_cod', 'deck_cod']

In [74]:
seed(a=8, version=2)

In [77]:
passengers_dic = [ {'Pclass':3, 'Sex_cod':1, 'Age':randint(0, 80), 'title_cod':titres_dic["Miss"], 'Embarked_cod' : randint(0, 3), 'deck_cod': randint(-1, 6)},
             {'Pclass':2, 'Sex_cod':1, 'Age':randint(0, 80), 'title_cod':titres_dic["Miss"], 'Embarked_cod' : randint(0, 3), 'deck_cod': randint(-1, 6)},
             {'Pclass':1, 'Sex_cod':1, 'Age':randint(0, 80), 'title_cod':titres_dic["Miss"], 'Embarked_cod' : randint(0, 3), 'deck_cod': randint(-1, 6)},
             {'Pclass':1, 'Sex_cod':1, 'Age':randint(15, 80), 'title_cod':titres_dic["Mrs"], 'Embarked_cod' : randint(0, 3), 'deck_cod': randint(-1, 6)},
             {'Pclass':2, 'Sex_cod':1, 'Age':randint(15, 80), 'title_cod':titres_dic["Mrs"], 'Embarked_cod' : randint(0, 3), 'deck_cod': randint(-1, 6)},
             {'Pclass':3, 'Sex_cod':1, 'Age':randint(15, 80), 'title_cod':titres_dic["Mrs"], 'Embarked_cod' : randint(0, 3), 'deck_cod': randint(-1, 6)},
             {'Pclass':3, 'Sex_cod':0, 'Age':randint(0, 80), 'title_cod':titres_dic["Mr"], 'Embarked_cod' : randint(0, 3), 'deck_cod': randint(-1, 6)},
             {'Pclass':2, 'Sex_cod':0, 'Age':randint(0, 80), 'title_cod':titres_dic["Mr"], 'Embarked_cod' : randint(0, 3), 'deck_cod': randint(-1, 6)},
             {'Pclass':1, 'Sex_cod':0, 'Age':randint(0, 80), 'title_cod':titres_dic["Mr"], 'Embarked_cod' : randint(0, 3), 'deck_cod': randint(-1, 6)},
             {'Pclass':1, 'Sex_cod':0, 'Age':randint(20, 80), 'title_cod':titres_dic["Master"], 'Embarked_cod' : randint(0, 3), 'deck_cod': randint(-1, 6)},
             {'Pclass':2, 'Sex_cod':0, 'Age':randint(20, 80), 'title_cod':titres_dic["Master"], 'Embarked_cod' : randint(0, 3), 'deck_cod': randint(-1, 6)},
             {'Pclass':3, 'Sex_cod':0, 'Age':randint(20, 80), 'title_cod':titres_dic["Master"], 'Embarked_cod' : randint(0, 3), 'deck_cod': randint(-1, 6)}
]

In [78]:
for passenger in passengers_dic:
    print('Pclass:', passenger['Pclass'], ", Sex_cod:", passenger['Sex_cod'], ', Age:',passenger['Age'], ', title_cod:', passenger['title_cod'], ', Embarked_cod:', passenger['Embarked_cod'], ', deck_cod:', passenger['deck_cod'] )

Pclass: 3 , Sex_cod: 1 , Age: 29 , title_cod: 6 , Embarked_cod: 2 , deck_cod: 5
Pclass: 2 , Sex_cod: 1 , Age: 16 , title_cod: 6 , Embarked_cod: 1 , deck_cod: -1
Pclass: 1 , Sex_cod: 1 , Age: 10 , title_cod: 6 , Embarked_cod: 1 , deck_cod: 2
Pclass: 1 , Sex_cod: 1 , Age: 79 , title_cod: 8 , Embarked_cod: 1 , deck_cod: 5
Pclass: 2 , Sex_cod: 1 , Age: 18 , title_cod: 8 , Embarked_cod: 3 , deck_cod: 6
Pclass: 3 , Sex_cod: 1 , Age: 73 , title_cod: 8 , Embarked_cod: 3 , deck_cod: 6
Pclass: 3 , Sex_cod: 0 , Age: 73 , title_cod: 7 , Embarked_cod: 1 , deck_cod: 5
Pclass: 2 , Sex_cod: 0 , Age: 11 , title_cod: 7 , Embarked_cod: 3 , deck_cod: 2
Pclass: 1 , Sex_cod: 0 , Age: 2 , title_cod: 7 , Embarked_cod: 2 , deck_cod: 5
Pclass: 1 , Sex_cod: 0 , Age: 50 , title_cod: 5 , Embarked_cod: 3 , deck_cod: 0
Pclass: 2 , Sex_cod: 0 , Age: 62 , title_cod: 5 , Embarked_cod: 2 , deck_cod: 0
Pclass: 3 , Sex_cod: 0 , Age: 72 , title_cod: 5 , Embarked_cod: 0 , deck_cod: 5


### KNN

In [ ]:
better_knn_model = better_models['kneighborsclassifier'][0]
better_knn_columns = better_models['kneighborsclassifier'][1]
print(better_knn_columns, better_knn_model)

['Pclass', 'Sex_cod'] GridSearchCV(cv=4,
             estimator=Pipeline(steps=[('kneighborsclassifier',
                                        KNeighborsClassifier())]),
             param_grid={'kneighborsclassifier__metric': ['minkowski'],
                         'kneighborsclassifier__n_neighbors': [1, 2, 3, 4, 5, 6,
                                                               7, 8, 9],
                         'kneighborsclassifier__p': [1, 2, 3, 4, 5, 6, 7, 8,
                                                     9]})


In [ ]:
@ignore_warnings(category=UserWarning)
def knn_predire_survie(model, pclass=3, sex=1, age=40, title=6, deck = randint(0, 6), embarked_cod = randint(-1, 2)):
    ['Pclass', 'Sex_cod']
    deck = randint(0, 6)
    embarked_cod = randint(-1, 2)
    # ['Pclass', 'sex_cod', 'title_cod', 'embarked_cod', 'deck_cod']
    params = [pclass, sex, title, embarked_cod, deck]
    return predire_survie(model, params)

In [ ]:
better_tree_model = better_models['decisiontreeclassifier'][0]
better_tree_columns = better_models['decisiontreeclassifier'][1]
print(better_tree_columns, better_tree_model)

['Embarked_cod', 'Sex_cod'] GridSearchCV(cv=4,
             estimator=Pipeline(steps=[('decisiontreeclassifier',
                                        DecisionTreeClassifier(random_state=0))]),
             param_grid={'decisiontreeclassifier__criterion': ['gini',
                                                               'entropy'],
                         'decisiontreeclassifier__splitter': ['best',
                                                              'random']})


In [56]:
better_forest_model = better_models['randomforestclassifier'][0]
better_forest_columns = better_models['randomforestclassifier'][1]
print(better_forest_columns, better_forest_model)

['Sex_cod', 'Titre_cod'] GridSearchCV(cv=4,
             estimator=Pipeline(steps=[('randomforestclassifier',
                                        RandomForestClassifier(random_state=0))]),
             param_grid={'randomforestclassifier__criterion': ['gini',
                                                               'entropy'],
                         'randomforestclassifier__n_estimators': [1, 11, 21, 31,
                                                                  41, 51, 61,
                                                                  71, 81, 91]})


In [57]:
better_logistic_model = better_models['logisticregression'][0]
better_logistic_columns = better_models['logisticregression'][1]
print(better_logistic_columns, better_logistic_model)

['Age', 'Embarked_cod', 'Sex_cod', 'Titre_cod'] GridSearchCV(cv=4,
             estimator=Pipeline(steps=[('logisticregression',
                                        LogisticRegression(random_state=0))]),
             param_grid={'logisticregression__fit_intercept': [True, False],
                         'logisticregression__penalty': [None, 'l2', 'l1',
                                                         'elasticnet'],
                         'logisticregression__solver': ['newton-cg', 'lbfgs',
                                                        'liblinear', 'sag',
                                                        'saga']})


## 5.3. RandomForestClassifier

* RandomForestClassifier
j'ai essayé le random forest sur ton fichier, j'ai eu un score de 0,85  et en ajoutant une colonne titre comme 'Countess,Major ...' j'arrive à avoir un score de 0,86
oui, avec les variables catégorielles comme le deck par exemple tu peux utiliser le from sklearn.preprocessing import OrdinalEncoder pour qu'elle soit exploitable par l'algorithme de machine learning
* KNeighborsClassifier

In [177]:
from sklearn.ensemble import RandomForestClassifier 

columns_logistic = ['Pclass', 'sex_cod', 'title_cod', 'embarked_cod', 'deck_cod']

model_forest = RandomForestClassifier(n_estimators=100,random_state=random_state)
model_forest.fit(X_train[columns_logistic],y_train)
pd.Series(model_forest.feature_importances_,index=columns_logistic).sort_values(ascending=False)

sex_cod         0.317152
deck_cod        0.226847
title_cod       0.206376
Pclass          0.159743
embarked_cod    0.089883
dtype: float64

In [178]:
model_forest.score(X_test[columns_logistic], y_test)

0.8770949720670391

## 5.4. Decision tree

In [179]:
columns_decision_tree_started = ['Pclass', 'sex_cod', 'title_cod', 'Age', 'family_on_board', 'Fare', 'embarked_cod', 'deck_cod']

model_decision_tree=DecisionTreeClassifier()
model_decision_tree.fit(X_train[columns_decision_tree_started],y_train)
print(model_decision_tree.score(X_test[columns_decision_tree_started], y_test))
prediction=model_decision_tree.predict(X_test[columns_decision_tree_started])
# print('The accuracy of the Decision Tree is',metrics.accuracy_score(prediction,test_y))

0.7988826815642458
